In [2]:
import os
import glob
import json
import re
from langchain_community.document_loaders import PyPDFLoader
from langchain_chroma import Chroma
from langchain.embeddings import OpenAIEmbeddings  # Или другой провайдер эмбеддингов
from langchain.docstore.document import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter

## Загрузка PDF файлов

In [3]:
import re, glob, os, json
from langchain.document_loaders import PyPDFLoader

def extract_sections(text: str):
    """
    Возвращает кортеж (фабула, решение, список статей).
    Фабула – всё, что между УСТАНОВИЛА: и заголовком решения.
    Решение – всё, что после первого заголовка решения.
    """

    # 1. Разбиваем по заголовку "УСТАНОВИЛА:"
    ust_re = re.compile(r'У\s*С\s*Т\s*А\s*Н\s*О\s*В\s*И\s*Л\s*А\s*:', re.IGNORECASE)
    parts = ust_re.split(text, maxsplit=1)
    if len(parts) == 2:
        after_ust = parts[1]
    else:
        # если нет "УСТАНОВИЛА:", считаем всю строку после – за единый блок
        after_ust = text

    # 2. Разбиваем по заголовку решения
    # Варианты: ОПРЕДЕЛИЛ(А), ПОСТАНОВИЛ(А), без учёта регистра и множества пробелов
    dec_re = re.compile(
        r'(?:О\s*П\s*Р\s*Е\s*Д\s*Е\s*Л\s*И\s*Л\s*А|'
        r'П\s*О\s*С\s*Т\s*А\s*Н\s*О\s*В\s*И\s*Л\s*А|'
        r'О\s*П\s*Р\s*Е\s*Д\s*Е\s*Л\s*И\s*Л|'
        r'П\s*О\s*С\s*Т\s*А\s*Н\s*О\s*В\s*И\s*Л)'
        r'\s*:', re.IGNORECASE | re.DOTALL
    )
    split = dec_re.split(after_ust, maxsplit=1)

    if len(split) == 2:
        fabula   = split[0].strip()
        decision = split[1].strip()
    else:
        fabula, decision = text.strip(), ''

    # 3. Извлечение статей
    seen = set()
    articles = []

    # 3.1 Диапазоны: "статями 284 – 289"
    range_re = re.compile(r'стать(?:я|и|ей|ями)\s+(\d+)\s*[-–—]\s*(\d+)', re.IGNORECASE)
    for m in range_re.finditer(text):
        start, end = int(m.group(1)), int(m.group(2))
        for i in range(start, end + 1):
            label = f'статья {i}'
            if label not in seen:
                seen.add(label)
                articles.append(label)

    # 3.2 Одиночные статьи и части
    article_re = re.compile(
        r'(?:част(?:ь|ью|и)\s*\d+\s+стать(?:я|и|ей|ями)\s*\d+)|'
        r'(?:стать(?:я|и|ей|ями)\s*\d+)',
        re.IGNORECASE
    )
    for m in article_re.finditer(text):
        raw = re.sub(r'\s+', ' ', m.group(0), flags=re.UNICODE).strip().lower()
        if raw not in seen:
            seen.add(raw)
            articles.append(raw)

    return fabula, decision, articles


def parse_pdf_and_save_json(directories, output_json_file):
    """
    Обходит PDF-файлы в указанных директориях,
    извлекает фабулу, решение и статьи для каждого,
    и сохраняет результаты в JSON-файл.
    """
    if isinstance(directories, str):
        directories = [directories]

    results = []
    for directory in directories:
        pdf_paths = glob.glob(os.path.join(directory, '**', '*.pdf'), recursive=True)
        print(f"В каталоге «{directory}» найдено {len(pdf_paths)} PDF-файлов.")
        for path in pdf_paths:
            try:
                loader = PyPDFLoader(path)
                docs = loader.load()
                full_text = "\n".join(doc.page_content for doc in docs)

                fabula, decision, articles = extract_sections(full_text)
                results.append({
                    'file': os.path.basename(path),
                    'фабула': fabula,
                    'решение': decision,
                    'статьи': articles
                })
            except Exception as e:
                print(f"Ошибка при обработке «{path}»: {e}")

    with open(output_json_file, 'w', encoding='utf-8') as f:
        json.dump(results, f, ensure_ascii=False, indent=4)

    print(f"Результаты сохранены в «{output_json_file}»")


if __name__ == "__main__":
    dirs = ["/Users/sergey/Desktop/RAG_Legal_asist/RAG_Legal_assistant"]
    out_json = "/Users/sergey/Desktop/RAG_Legal_asist/RAG_Legal_assistant/Data/parsed_output.json"
    parse_pdf_and_save_json(dirs, out_json)

В каталоге «/Users/sergey/Desktop/RAG_Legal_asist/RAG_Legal_assistant» найдено 1 PDF-файлов.
Результаты сохранены в «/Users/sergey/Desktop/RAG_Legal_asist/RAG_Legal_assistant/Data/parsed_output.json»


## Загружаем переменные окружения

In [4]:
from dotenv import load_dotenv

# Загружаем переменные из файла .env
load_dotenv()

# Теперь переменные доступны в os.environ
openai_api_key = os.getenv("OPENAI_API_KEY")

if openai_api_key:
    print("OpenAI API Key успешно загружен.")
else:
    print("Ошибка: OpenAI API Key не найден.")

OpenAI API Key успешно загружен.


## Загружаем распарсиный документ

In [5]:
extracted_document = "/Users/sergey/Desktop/RAG_Legal_asist/RAG_Legal_assistant/Data/parsed_output.json"

with open(extracted_document, 'r', encoding='utf-8') as f:
    extracted_document = json.load(f)

In [6]:
import os
import json
from langchain.schema import Document  # или: from langchain.docstore.document import Document

def load_extracted_document(json_path):
    """
    Загружает JSON с результатами парсинга (один документ) и конвертирует его в Document.
    Возвращает либо сам Document, либо список из одного элемента, если нужно.
    """
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # Если в JSON лежит список, берём первый элемент
    if isinstance(data, list):
        item = data[0]
    else:
        item = data

    # Собираем метаданные
    metadata = {
        'file': item.get('file'),
        'статьи': item.get('статьи', []),
    }

    # Если явного title нет, используем имя файла без расширения
    if not metadata.get('title') and metadata.get('file'):
        metadata['title'] = os.path.splitext(metadata['file'])[0]

    # Формируем содержимое документа
    fabula   = item.get('фабула', '').strip()
    decision = item.get('решение', '').strip()
    page_content = '\n\n'.join([
        'Фабула:',
        fabula,
        '--',
        'Решение:',
        decision
    ])

    # Создаём Document
    doc = Document(
        page_content=page_content,
        metadata=metadata
    )
    return doc

if __name__ == "__main__":
    json_path = "/Users/sergey/Desktop/RAG_Legal_asist/RAG_Legal_assistant/Data/parsed_output.json"
    document = load_extracted_document(json_path)
    print("Сконвертирован Document:")
    print("page_content:", document.page_content)
    print("metadata:", document.metadata)

Сконвертирован Document:
page_content: Фабула:

106462_3301877 
  
 
 
 АРБИТРАЖНЫЙ СУД 
МОСКОВСКОГО ОКРУГА 
 
ул. Селезнёвская, д. 9, г. Москва, ГСП-4, 127994,  
официальный сайт: http://www.fasmo.arbitr.ru e-mail: info@fasmo.arbitr.ru 
 
П О С ТАН О В ЛЕ Н И Е 
г. Москва  
20 мая 2025 года   
 
                               Дело № А41-9245/2025 
 
Резолютивная часть постановления объявлена 13 мая 2025 года  
Полный текст постановления изготовлен 20 мая 2025 года  
  
Арбитражный суд Московского округа 
в составе:  
председательствующего судьи Нечаева С.В., 
судей Каденковой Е.Г., Кочеткова А.А.,  
при участии в судебном заседании: 
от истца: лично (онлайн) 
рассмотрев в  судебном заседании кассационную жалобу 
ИП Фоменко Алексея Сергеевича 
на определение Десятого арбитражного апелляционного суда от 13 марта 
2025 года о возвращении апелляционной жалобы, 
по делу по иску ИП Фоменко Алексея Сергеевича 
к Арбитражному суду Белгородской области 
о взыскании морального вреда 
 УСТАНОВИЛ

In [7]:
document

Document(metadata={'file': 'A41-9245-2025_20250520_Reshenija_i_postanovlenija.pdf', 'статьи': ['статья 284', 'статья 285', 'статья 286', 'статья 287', 'статья 288', 'статья 289', 'статьи 286', 'статьи 17', 'статьи 264', 'части 5 статьи 264', 'частью 4 статьи 288', 'статьями 284'], 'title': 'A41-9245-2025_20250520_Reshenija_i_postanovlenija'}, page_content='Фабула:\n\n106462_3301877 \n  \n \n \n АРБИТРАЖНЫЙ СУД \nМОСКОВСКОГО ОКРУГА \n \nул. Селезнёвская, д. 9, г. Москва, ГСП-4, 127994,  \nофициальный сайт: http://www.fasmo.arbitr.ru e-mail: info@fasmo.arbitr.ru \n \nП О С ТАН О В ЛЕ Н И Е \nг. Москва  \n20 мая 2025 года   \n \n                               Дело № А41-9245/2025 \n \nРезолютивная часть постановления объявлена 13 мая 2025 года  \nПолный текст постановления изготовлен 20 мая 2025 года  \n  \nАрбитражный суд Московского округа \nв составе:  \nпредседательствующего судьи Нечаева С.В., \nсудей Каденковой Е.Г., Кочеткова А.А.,  \nпри участии в судебном заседании: \nот истца: л

## Создание разбиения на чанки с помощью RecursiveCharacterTextSplitter.

In [8]:
# Шаг 1: Настройка текстового сплиттера
text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)

# Шаг 2: Применение разбиения к единственному документу
recursive_chunks = []
chunks = text_splitter.split_text(document.page_content)
for chunk in chunks:
    recursive_chunks.append(Document(page_content=chunk, metadata=document.metadata))

# Шаг 3: Сохранение результатов в JSON-файл
output_dir = "/Users/sergey/Desktop/RAG_Legal_asist/RAG_Legal_assistant/Data"
os.makedirs(output_dir, exist_ok=True)

output_file = os.path.join(output_dir, "recursive_chunks.json")

serialized_docs = [
    {"page_content": doc.page_content, "metadata": doc.metadata}
    for doc in recursive_chunks
]

with open(output_file, "w", encoding="utf-8") as f:
    json.dump(serialized_docs, f, ensure_ascii=False, indent=4)

print(f"Результаты разбиения сохранены в файл: {output_file}")

Результаты разбиения сохранены в файл: /Users/sergey/Desktop/RAG_Legal_asist/RAG_Legal_assistant/Data/recursive_chunks.json


In [9]:
len(serialized_docs)

26

## Просмотр того как выполнено разбиение 

In [10]:
for i, doc in enumerate(serialized_docs[:3], start=1):
    print(f"Часть {i}:")
    print(f"Содержание:\n{doc['page_content']}\n")
    print(f"Метаданные: {doc['metadata']}\n")
    print("-" * 80)

Часть 1:
Содержание:
Фабула:

Метаданные: {'file': 'A41-9245-2025_20250520_Reshenija_i_postanovlenija.pdf', 'статьи': ['статья 284', 'статья 285', 'статья 286', 'статья 287', 'статья 288', 'статья 289', 'статьи 286', 'статьи 17', 'статьи 264', 'части 5 статьи 264', 'частью 4 статьи 288', 'статьями 284'], 'title': 'A41-9245-2025_20250520_Reshenija_i_postanovlenija'}

--------------------------------------------------------------------------------
Часть 2:
Содержание:
106462_3301877 
  
 
 
 АРБИТРАЖНЫЙ СУД 
МОСКОВСКОГО ОКРУГА 
 
ул. Селезнёвская, д. 9, г. Москва, ГСП-4, 127994,  
официальный сайт: http://www.fasmo.arbitr.ru e-mail: info@fasmo.arbitr.ru 
 
П О С ТАН О В ЛЕ Н И Е 
г. Москва  
20 мая 2025 года   
 
                               Дело № А41-9245/2025 
 
Резолютивная часть постановления объявлена 13 мая 2025 года

Метаданные: {'file': 'A41-9245-2025_20250520_Reshenija_i_postanovlenija.pdf', 'статьи': ['статья 284', 'статья 285', 'статья 286', 'статья 287', 'статья 288', 'ста

## Создание векторной базы данных ChromaDB

In [11]:
# Шаг 1: Загрузка чанков из JSON
input_file = "//Users/sergey/Desktop/RAG_Legal_asist/RAG_Legal_assistant/Data/recursive_chunks.json"
with open(input_file, "r", encoding="utf-8") as f:
    serialized = json.load(f)

# 2) Конвертация в Document с примитивными метаданными
documents = []
for item in serialized:
    raw_meta = item["metadata"]
    # приводим список статей к строке
    articles = raw_meta.get("статьи")
    if isinstance(articles, list):
        raw_meta["статьи"] = ", ".join(articles)
    documents.append(
        Document(page_content=item["page_content"], metadata=raw_meta)
    )

# Шаг 2: Настройка эмбеддингов
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

# Шаг 3: Инициализация и наполнение Chroma DB
persist_directory = "/Users/sergey/Desktop/RAG_Legal_asist/RAG_Legal_assistant/Data/chroma_db/recursive_chunks"
os.makedirs(persist_directory, exist_ok=True)

vectordb = Chroma(
    collection_name="recursive_chunks",
    embedding_function=embeddings,
    persist_directory=persist_directory
)

# 5) Добавление всех документов
vectordb.add_documents(documents)
print(f"Добавлено {len(documents)} документов в коллекцию 'recursive_chunks'.")

Добавлено 26 документов в коллекцию 'recursive_chunks'.


## Загружаем базу данных из указаной директории

In [12]:
# Укажите директорию, где сохранена база данных Chroma
persist_directory = "/Users/sergey/Desktop/RAG_Legal_asist/RAG_Legal_assistant/Data/chroma_db/recursive_chunks"

# Настройка провайдера эмбеддингов
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

# Шаг 1: Загрузка базы данных Chroma
vectordb = Chroma(
    collection_name="recursive_chunks",
    persist_directory=persist_directory,
    embedding_function=embeddings
)

# Проверка количества документов в базе данных
doc_count = len(vectordb._collection.get()["documents"])
print(f"Количество документов в базе данных: {doc_count}")

Количество документов в базе данных: 26


## Определяем способ поиска и колличество возвращаемых документов

In [13]:
# Инициализируем retriever с типом поиска по схожести, чтобы получать k=3 наиболее релевантных результатов
retriever = vectordb.as_retriever(
    search_type="similarity",  # Используем метод поиска по схожести
    search_kwargs={"k": 5},  # Количество возвращаемых результатов: 3
)

In [14]:
# 1) Инициализируем LLM
llm = ChatOpenAI(model="gpt-4o-mini")

# 2) Системный промпт для контекстуализации вопроса
contextualize_q_system_prompt = (
    "Сформулируйте самодостаточный вопрос, который можно понять самостоятельно. "
    "НЕ отвечайте на вопрос, просто переформулируйте его, "
    "или верните его без изменений."
)
contextualize_q_prompt = ChatPromptTemplate.from_messages([
    ("system", contextualize_q_system_prompt),
    ("human", "{input}")
])

# 3) Оборачиваем retriever в history-aware с нашим prompt
history_aware_retriever = create_history_aware_retriever(
    llm,
    retriever,
    contextualize_q_prompt
)

# 4) Системный промпт для ответов на вопросы
qa_system_prompt = (
    "Вы юридический помощник, обученный отвечать на вопросы на основании судебных постановлений."

    "Ваша задача — предоставить чёткий, краткий и обоснованный ответ на вопрос пользователя, используя исключительно переданный контекст из базы данных."
    "Если информации недостаточно, ответьте: 'Таких данных нет в базе данных.' "

    "Структура ответа должна быть строго следующей, с обязательными переносами строк перед каждым блоком:"

    "<сформулированный ответ>"

    "Актуальное решение суда:"  
    "<краткое изложение сути судебного решения, если оно присутствует в контексте>"

    "Основание для решения:"  
    "- <название суда>"  
    "- <номер дела>"  
    "- <дата и наименование постановления>"  
    "- <статьи и нормы права, если они присутствуют>"

    "{context}"
)
qa_prompt = ChatPromptTemplate.from_messages([
    ("system", qa_system_prompt),
    ("human", "{input}")
])

# 6) Строим RAG-цепочку
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)
contextualize_chain = contextualize_q_prompt | llm

# 7) Функция для интерактивного чата с историей
def continual_chat():
    print("Начните общение с ИИ! Введите ‘выход’, чтобы завершить разговор.")
    chat_history = []

    while True:
        query = input("ВЫ: ")
        if query.lower() == "выход":
            break

        # Явно вызываем переформулировку
        rephrased = contextualize_chain.invoke({"input": query}).content.strip()

        # Вывод вопросов
        print("\n🔹 Оригинальный вопрос:")
        print(query)
        if rephrased != query:
            print("\n🔸 Перефразированный вопрос:")
            print(rephrased)

        # Вызываем цепочку RAG с перефразированным вопросом
        result = rag_chain.invoke({
            "input": rephrased,
            "chat_history": chat_history
        })

        # Вывод ответа
        print("\n✅ Ответ AI:")
        print(result['answer'])

        # Обновление истории (с оригинальным вопросом)
        chat_history.append(HumanMessage(content=query))
        chat_history.append(SystemMessage(content=result["answer"]))

# 8) Запуск интерактивного чата
if __name__ == "__main__":
    continual_chat()

Начните общение с ИИ! Введите ‘выход’, чтобы завершить разговор.

🔹 Оригинальный вопрос:
Кто истец по делу?

🔸 Перефразированный вопрос:
Кто является истцом в данном судебном деле?

✅ Ответ AI:
Истцом в данном судебном деле является ИП Фоменко Алексей Сергеевич.  

Актуальное решение суда: 
Определением Арбитражного суда Московской области от 12 февраля 2025 года исковое заявление возвращено в связи с отсутствием у арбитражного суда компетенции по рассмотрению настоящего спора.  

Основание для решения:  
- Арбитражный суд Московской области  
- № А41-9245/2025  
- 12 февраля 2025 года, определение  
- нет данных о статьях и нормах права.  

Полный текст постановления изготовлен 20 мая 2025 года.

🔹 Оригинальный вопрос:
Что решил суд?

🔸 Перефразированный вопрос:
Какое решение принял суд?

✅ Ответ AI:
Суд оставил определение Десятого арбитражного апелляционного суда от 13 марта 2025 года по делу № А41-9245/2025 без изменения, а кассационную жалобу – без удовлетворения.

Актуальное реше